In [1]:
import sys

sys.path.append("../")
from calculus_path_mod.term_engine import *
from calculus_path_mod.reduction_strategy import *

Lambda = Abstraction
App = Application

# <span style="color: #A1F99C">TEST PROCEDURE</span>

In [2]:
rs = RandomOuterStrategy()


def test_procedure(term, reduction_strategy=rs, print_steps=True,
                   steps_lim=60, assert_result=""):
    term = term._update_bound_vars()
    n_steps = 0
    is_term_normalized = True
    if print_steps:
        print(f"{n_steps}: {term}")
    while term.redexes:
        term = term._beta_conversion(reduction_strategy)
        term = term._update_bound_vars()
        n_steps += 1
        if print_steps:
            print(f"{n_steps}: {term}")
        if n_steps > steps_lim:
            is_term_normalized = False
            break

    print(f"\n\nreduction steps: {n_steps}")
    print(f"norm term:       {term}" if is_term_normalized else f"NOT norm term: {term}")

# <span style="color: #A1F99C">USEFUL TERMS</span>

In [3]:
def ite_term():
    x, y, c = Var(), Var(), Var()
    x_, y_, c_ = Atom(x), Atom(y), Atom(c)
    return Lambda(c, Lambda(x, Lambda(y, multi_app_term(c_, x_, y_))))


def true_term():
    x, y = Var(), Var()
    x_ = Atom(x)
    return Lambda(x, Lambda(y, x_))


def num_zero_term():
    s, z = Var(), Var()
    z_ = Atom(z)
    return Lambda(s, Lambda(z, z_))


def num_term(n: int):
    if n <= 0:
        return num_zero_term()
    s, z = Var(), Var()
    s_, z_ = Atom(s), Atom(z)
    core_term = App(s_, z_)
    for _ in range(n - 1):
        core_term = App(s_, core_term)
    return Lambda(s, Lambda(z, core_term))


def multi_app_term(term_0: Term, term_1: Term, *terms: Term):
    res_app_term = App(term_0, term_1)
    for term in terms:
        res_app_term = App(res_app_term, term)
    return res_app_term


def false_term():
    x, y = Var(), Var()
    y_ = Atom(y)
    return Lambda(x, Lambda(y, y_))


def pair_term():
    x, y, p = Var(), Var(), Var()
    x_, y_, p_ = Atom(x), Atom(y), Atom(p)
    return Lambda(x, Lambda(y, Lambda(p, multi_app_term(p_, x_, y_))))


def first_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, true_term()))


def second_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(p, App(p_, false_term()))


def succ_term():
    x, y, n = Var(), Var(), Var()
    x_, y_, n_ = Atom(x), Atom(y), Atom(n)
    return Lambda(n, Lambda(x, Lambda(y, App(x_, multi_app_term(n_, x_, y_)))))


def sinc_term():
    p = Var()
    p_ = Atom(p)
    return Lambda(
        p,
        multi_app_term(
            pair_term(),
            App(second_term(), p_),
            App(succ_term(), App(second_term(), p_)),
        ),
    )


def pred_term():
    n, f, x, g, h, u = Var(), Var(), Var(), Var(), Var(), Var()
    n_, f_, x_ = Atom(n), Atom(f), Atom(x)
    g_, h_, u_ = Atom(g), Atom(h), Atom(u)

    return Lambda(n, Lambda(f, Lambda(x, multi_app_term(
        n_,
        Lambda(g, Lambda(h, App(h_, App(g_, f_)))),
        Lambda(u, x_),
        Lambda(u, u_)
    ))))


def plus_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(m, Lambda(n, multi_app_term(n_, succ_term(), m_)))


def subtract_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(n, Lambda(m, multi_app_term(m_, pred_term(), n_)))


def iszero_term():
    x, n = Var(), Var()
    n_ = Atom(n)
    return Lambda(n, App(App(n_, Lambda(x, false_term())), true_term()))


def leq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n, Lambda(m, App(iszero_term(), multi_app_term(subtract_term(), n_, m_)))
    )


def and_term():
    a, b = Var(), Var()
    a_, b_ = Atom(a), Atom(b)
    return Lambda(a, Lambda(b, multi_app_term(ite_term(), a_, b_, a_)))


def eq_term():
    n, m = Var(), Var()
    n_, m_ = Atom(n), Atom(m)
    return Lambda(
        n,
        Lambda(
            m,
            multi_app_term(
                and_term(),
                multi_app_term(leq_term(), n_, m_),
                multi_app_term(leq_term(), m_, n_),
            ),
        ),
    )

# <span style="color: #A1F99C">1.</span>
<span style="color:#bb91f3">(λx. (λy.y)) ((λx. (xx)) (λx. (xx)))</span> == <span style="color:#f1bd47">I</span> == <span style="color:#f1bd47">λx. x</span>
<span style="color:#c9fafa">This term reduce in one step by LO, RO strategy
And reduce in the same terms with other strategy.</span>

In [4]:
def term_test_1():
    x, y = Var(), Var()
    x_, y_ = Atom(x), Atom(y)

    return App(
             Lambda(x, Lambda(y, y_)),
             App(
                 Lambda(x, App(x_, x_)),
                 Lambda(x, App(x_, x_))
             )
         )


test_procedure(term_test_1())

0: ((λ#2. (λ#3. #3)) ((λ#4. (#4 #4)) (λ#5. (#5 #5))))
count_redexes: 2
1: (λ#6. #6)


reduction steps: 1
norm term:       (λ#6. #6)


# <span style="color: #A1F99C">2.</span>
<span style="color:#bb91f3">(ITE TRUE 3 1)</span> == <span style="color:#f1bd47>">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [5]:
def term_test_2():
    return multi_app_term(ite_term(), true_term(), num_term(3), num_term(1))


test_procedure(term_test_2())

0: ((((λ#16. (λ#17. (λ#18. ((#16 #17) #18)))) (λ#19. (λ#20. #19))) (λ#21. (λ#22. (#21 (#21 (#21 #22)))))) (λ#23. (λ#24. (#23 #24))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
1: (((λ#25. (λ#26. (((λ#27. (λ#28. #27)) #25) #26))) (λ#29. (λ#30. (#29 (#29 (#29 #30)))))) (λ#31. (λ#32. (#31 #32))))
count_redexes: 2
count_redexes: 2
2: ((λ#33. (((λ#34. (λ#35. #34)) (λ#36. (λ#37. (#36 (#36 (#36 #37)))))) #33)) (λ#38. (λ#39. (#38 #39))))
count_redexes: 2
3: (((λ#40. (λ#41. #40)) (λ#42. (λ#43. (#42 (#42 (#42 #43)))))) (λ#44. (λ#45. (#44 #45))))
count_redexes: 1
count_redexes: 1
4: ((λ#46. (λ#47. (λ#48. (#47 (#47 (#47 #48)))))) (λ#49. (λ#50. (#49 #50))))
count_redexes: 1
5: (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


reduction steps: 5
norm term:       (λ#51. (λ#52. (#51 (#51 (#51 #52)))))


# <span style="color: #A1F99C">3.</span>
<span style="color:#bb91f3">(ITE FALSE 0 4)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [6]:
def term_test_3():
    return multi_app_term(ite_term(), false_term(), num_term(0), num_term(4))


test_procedure(term_test_3())

0: ((((λ#62. (λ#63. (λ#64. ((#62 #63) #64)))) (λ#65. (λ#66. #66))) (λ#67. (λ#68. #68))) (λ#69. (λ#70. (#69 (#69 (#69 (#69 #70)))))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
1: (((λ#71. (λ#72. (((λ#73. (λ#74. #74)) #71) #72))) (λ#75. (λ#76. #76))) (λ#77. (λ#78. (#77 (#77 (#77 (#77 #78)))))))
count_redexes: 2
count_redexes: 2
2: ((λ#79. (((λ#80. (λ#81. #81)) (λ#82. (λ#83. #83))) #79)) (λ#84. (λ#85. (#84 (#84 (#84 (#84 #85)))))))
count_redexes: 2
3: (((λ#86. (λ#87. #87)) (λ#88. (λ#89. #89))) (λ#90. (λ#91. (#90 (#90 (#90 (#90 #91)))))))
count_redexes: 1
count_redexes: 1
4: ((λ#92. #92) (λ#93. (λ#94. (#93 (#93 (#93 (#93 #94)))))))
count_redexes: 1
5: (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


reduction steps: 5
norm term:       (λ#95. (λ#96. (#95 (#95 (#95 (#95 #96))))))


# <span style="color: #A1F99C">4.</span>
<span style="color:#bb91f3">(PRED 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [7]:
def term_test_4():
    return multi_app_term(pred_term(), num_term(0))


test_procedure(term_test_4())

0: ((λ#105. (λ#106. (λ#107. (((#105 (λ#108. (λ#109. (#109 (#108 #106))))) (λ#110. #107)) (λ#111. #111))))) (λ#112. (λ#113. #113)))
count_redexes: 1
1: (λ#114. (λ#115. ((((λ#116. (λ#117. #117)) (λ#118. (λ#119. (#119 (#118 #114))))) (λ#120. #115)) (λ#121. #121))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (λ#122. (λ#123. (((λ#124. #124) (λ#125. #123)) (λ#126. #126))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
3: (λ#127. (λ#128. ((λ#129. #128) (λ#130. #130))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
4: (λ#131. (λ#132. #132))


reduction steps: 4
norm term:       (λ#131. (λ#132. #132))


# <span style="color: #A1F99C">5.</span>
<span style="color:#bb91f3">(PRED 5)</span> == <span style="color:#f1bd47">4</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x y))))</span>

In [8]:
def term_test_5():
    return multi_app_term(pred_term(), num_term(5))


test_procedure(term_test_5())

0: ((λ#141. (λ#142. (λ#143. (((#141 (λ#144. (λ#145. (#145 (#144 #142))))) (λ#146. #143)) (λ#147. #147))))) (λ#148. (λ#149. (#148 (#148 (#148 (#148 (#148 #149))))))))
count_redexes: 1
1: (λ#150. (λ#151. ((((λ#152. (λ#153. (#152 (#152 (#152 (#152 (#152 #153))))))) (λ#154. (λ#155. (#155 (#154 #150))))) (λ#156. #151)) (λ#157. #157))))
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
count_redexes: 1
2: (λ#158. (λ#159. (((λ#160. ((λ#161. (λ#162. (#162 (#161 #158)))) ((λ#163. (λ#164. (#164 (#163 #158)))) ((λ#165. (λ#166. (#166 (#165 #158)))) ((λ#167. (λ#168. (#168 (#167 #158)))) ((λ#169. (λ#170. (#170 (#169 #158)))) #160)))))) (λ#171. #159)) (λ#172. #172))))
count_redexes: 6
count_redexes: 6
count_redexes: 6
count_redexes: 6
3: (λ#173. (λ#174. (((λ#175. (λ#176. (#176 (#175 #173)))) ((λ#177. (λ#178. (#178 (#177 #173)))) ((λ#179. (λ#180. (#180 (#179 #173)))) ((λ#181. (λ#182. (#182 (#181 #173)))) ((λ#183. (λ#184. (#184 (#183 #173)))) (λ#185. #174)))))) (λ#186. #186))))
count_

# <span style="color: #A1F99C">6.</span>
<span style="color:#bb91f3">(PLUS 0 0)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [9]:
def term_test_6():
    return multi_app_term(plus_term(), num_term(0), num_term(0))


test_procedure(term_test_6())

0: (((λ#286. (λ#287. ((#287 (λ#288. (λ#289. (λ#290. (#289 ((#288 #289) #290)))))) #286))) (λ#291. (λ#292. #292))) (λ#293. (λ#294. #294)))
count_redexes: 1
count_redexes: 1
1: ((λ#295. ((#295 (λ#296. (λ#297. (λ#298. (#297 ((#296 #297) #298)))))) (λ#299. (λ#300. #300)))) (λ#301. (λ#302. #302)))
count_redexes: 1
2: (((λ#303. (λ#304. #304)) (λ#305. (λ#306. (λ#307. (#306 ((#305 #306) #307)))))) (λ#308. (λ#309. #309)))
count_redexes: 1
count_redexes: 1
3: ((λ#310. #310) (λ#311. (λ#312. #312)))
count_redexes: 1
4: (λ#313. (λ#314. #314))


reduction steps: 4
norm term:       (λ#313. (λ#314. #314))


# <span style="color: #A1F99C">7.</span>
<span style="color:#bb91f3">(PLUS 2 3)</span> == <span style="color:#f1bd47">5</span> == <span style="color:#f1bd47">λx. λy. (x (x (x (x (x y)))))</span>

In [10]:
def term_test_7():
    return multi_app_term(plus_term(), num_term(2), num_term(3))


test_procedure(term_test_7())

0: (((λ#324. (λ#325. ((#325 (λ#326. (λ#327. (λ#328. (#327 ((#326 #327) #328)))))) #324))) (λ#329. (λ#330. (#329 (#329 #330))))) (λ#331. (λ#332. (#331 (#331 (#331 #332))))))
count_redexes: 1
count_redexes: 1
1: ((λ#333. ((#333 (λ#334. (λ#335. (λ#336. (#335 ((#334 #335) #336)))))) (λ#337. (λ#338. (#337 (#337 #338)))))) (λ#339. (λ#340. (#339 (#339 (#339 #340))))))
count_redexes: 1
2: (((λ#341. (λ#342. (#341 (#341 (#341 #342))))) (λ#343. (λ#344. (λ#345. (#344 ((#343 #344) #345)))))) (λ#346. (λ#347. (#346 (#346 #347)))))
count_redexes: 1
count_redexes: 1
3: ((λ#348. ((λ#349. (λ#350. (λ#351. (#350 ((#349 #350) #351))))) ((λ#352. (λ#353. (λ#354. (#353 ((#352 #353) #354))))) ((λ#355. (λ#356. (λ#357. (#356 ((#355 #356) #357))))) #348)))) (λ#358. (λ#359. (#358 (#358 #359)))))
count_redexes: 4
4: ((λ#360. (λ#361. (λ#362. (#361 ((#360 #361) #362))))) ((λ#363. (λ#364. (λ#365. (#364 ((#363 #364) #365))))) ((λ#366. (λ#367. (λ#368. (#367 ((#366 #367) #368))))) (λ#369. (λ#370. (#369 (#369 #370)))))))
c

# <span style="color: #A1F99C">8.</span>
<span style="color:#bb91f3">(SUBTRACT 4 1)</span> == <span style="color:#f1bd47">3</span> == <span style="color:#f1bd47">λx. λy. (x (x (x y)))</span>

In [11]:
def term_test_8():
    return multi_app_term(subtract_term(), num_term(4), num_term(1))


test_procedure(term_test_8())

0: (((λ#437. (λ#438. ((#438 (λ#439. (λ#440. (λ#441. (((#439 (λ#442. (λ#443. (#443 (#442 #440))))) (λ#444. #441)) (λ#445. #445)))))) #437))) (λ#446. (λ#447. (#446 (#446 (#446 (#446 #447))))))) (λ#448. (λ#449. (#448 #449))))
count_redexes: 1
count_redexes: 1
1: ((λ#450. ((#450 (λ#451. (λ#452. (λ#453. (((#451 (λ#454. (λ#455. (#455 (#454 #452))))) (λ#456. #453)) (λ#457. #457)))))) (λ#458. (λ#459. (#458 (#458 (#458 (#458 #459)))))))) (λ#460. (λ#461. (#460 #461))))
count_redexes: 1
2: (((λ#462. (λ#463. (#462 #463))) (λ#464. (λ#465. (λ#466. (((#464 (λ#467. (λ#468. (#468 (#467 #465))))) (λ#469. #466)) (λ#470. #470)))))) (λ#471. (λ#472. (#471 (#471 (#471 (#471 #472)))))))
count_redexes: 1
count_redexes: 1
3: ((λ#473. ((λ#474. (λ#475. (λ#476. (((#474 (λ#477. (λ#478. (#478 (#477 #475))))) (λ#479. #476)) (λ#480. #480))))) #473)) (λ#481. (λ#482. (#481 (#481 (#481 (#481 #482)))))))
count_redexes: 2
4: ((λ#483. (λ#484. (λ#485. (((#483 (λ#486. (λ#487. (#487 (#486 #484))))) (λ#488. #485)) (λ#489. #489)

# <span style="color: #A1F99C">9.</span>
<span style="color:#bb91f3">(SUBTRACT 3 5)</span> == <span style="color:#f1bd47">0</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [12]:
def term_test_9():
    return multi_app_term(subtract_term(), num_term(3), num_term(5))


test_procedure(term_test_9())

0: (((λ#602. (λ#603. ((#603 (λ#604. (λ#605. (λ#606. (((#604 (λ#607. (λ#608. (#608 (#607 #605))))) (λ#609. #606)) (λ#610. #610)))))) #602))) (λ#611. (λ#612. (#611 (#611 (#611 #612)))))) (λ#613. (λ#614. (#613 (#613 (#613 (#613 (#613 #614))))))))
count_redexes: 1
count_redexes: 1
1: ((λ#615. ((#615 (λ#616. (λ#617. (λ#618. (((#616 (λ#619. (λ#620. (#620 (#619 #617))))) (λ#621. #618)) (λ#622. #622)))))) (λ#623. (λ#624. (#623 (#623 (#623 #624))))))) (λ#625. (λ#626. (#625 (#625 (#625 (#625 (#625 #626))))))))
count_redexes: 1
2: (((λ#627. (λ#628. (#627 (#627 (#627 (#627 (#627 #628))))))) (λ#629. (λ#630. (λ#631. (((#629 (λ#632. (λ#633. (#633 (#632 #630))))) (λ#634. #631)) (λ#635. #635)))))) (λ#636. (λ#637. (#636 (#636 (#636 #637))))))
count_redexes: 1
count_redexes: 1
3: ((λ#638. ((λ#639. (λ#640. (λ#641. (((#639 (λ#642. (λ#643. (#643 (#642 #640))))) (λ#644. #641)) (λ#645. #645))))) ((λ#646. (λ#647. (λ#648. (((#646 (λ#649. (λ#650. (#650 (#649 #647))))) (λ#651. #648)) (λ#652. #652))))) ((λ#653. (λ

# <span style="color: #A1F99C">10.</span>
<span style="color:#bb91f3">(LEQ 3 2)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [13]:
def term_test_10():
    return multi_app_term(leq_term(), num_term(3), num_term(2))


test_procedure(term_test_10())

0: (((λ#1759. (λ#1760. ((λ#1761. ((#1761 (λ#1762. (λ#1763. (λ#1764. #1764)))) (λ#1765. (λ#1766. #1765)))) (((λ#1767. (λ#1768. ((#1768 (λ#1769. (λ#1770. (λ#1771. (((#1769 (λ#1772. (λ#1773. (#1773 (#1772 #1770))))) (λ#1774. #1771)) (λ#1775. #1775)))))) #1767))) #1759) #1760)))) (λ#1776. (λ#1777. (#1776 (#1776 (#1776 #1777)))))) (λ#1778. (λ#1779. (#1778 (#1778 #1779)))))
count_redexes: 3
count_redexes: 3
1: ((λ#1780. ((λ#1781. ((#1781 (λ#1782. (λ#1783. (λ#1784. #1784)))) (λ#1785. (λ#1786. #1785)))) (((λ#1787. (λ#1788. ((#1788 (λ#1789. (λ#1790. (λ#1791. (((#1789 (λ#1792. (λ#1793. (#1793 (#1792 #1790))))) (λ#1794. #1791)) (λ#1795. #1795)))))) #1787))) (λ#1796. (λ#1797. (#1796 (#1796 (#1796 #1797)))))) #1780))) (λ#1798. (λ#1799. (#1798 (#1798 #1799)))))
count_redexes: 3
2: ((λ#1800. ((#1800 (λ#1801. (λ#1802. (λ#1803. #1803)))) (λ#1804. (λ#1805. #1804)))) (((λ#1806. (λ#1807. ((#1807 (λ#1808. (λ#1809. (λ#1810. (((#1808 (λ#1811. (λ#1812. (#1812 (#1811 #1809))))) (λ#1813. #1810)) (λ#1814. #1814)

# <span style="color: #A1F99C">11.</span>
<span style="color:#bb91f3">(LEQ 2 5)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [14]:
def term_test_11():
    return multi_app_term(leq_term(), num_term(2), num_term(5))


test_procedure(term_test_11())

0: (((λ#2341. (λ#2342. ((λ#2343. ((#2343 (λ#2344. (λ#2345. (λ#2346. #2346)))) (λ#2347. (λ#2348. #2347)))) (((λ#2349. (λ#2350. ((#2350 (λ#2351. (λ#2352. (λ#2353. (((#2351 (λ#2354. (λ#2355. (#2355 (#2354 #2352))))) (λ#2356. #2353)) (λ#2357. #2357)))))) #2349))) #2341) #2342)))) (λ#2358. (λ#2359. (#2358 (#2358 #2359))))) (λ#2360. (λ#2361. (#2360 (#2360 (#2360 (#2360 (#2360 #2361))))))))
count_redexes: 3
count_redexes: 3
1: ((λ#2362. ((λ#2363. ((#2363 (λ#2364. (λ#2365. (λ#2366. #2366)))) (λ#2367. (λ#2368. #2367)))) (((λ#2369. (λ#2370. ((#2370 (λ#2371. (λ#2372. (λ#2373. (((#2371 (λ#2374. (λ#2375. (#2375 (#2374 #2372))))) (λ#2376. #2373)) (λ#2377. #2377)))))) #2369))) (λ#2378. (λ#2379. (#2378 (#2378 #2379))))) #2362))) (λ#2380. (λ#2381. (#2380 (#2380 (#2380 (#2380 (#2380 #2381))))))))
count_redexes: 3
count_redexes: 2
count_redexes: 2
2: ((λ#2382. (((((λ#2383. (λ#2384. ((#2384 (λ#2385. (λ#2386. (λ#2387. (((#2385 (λ#2388. (λ#2389. (#2389 (#2388 #2386))))) (λ#2390. #2387)) (λ#2391. #2391))))))

# <span style="color: #A1F99C">12.</span>
<span style="color:#bb91f3">(EQ 2 2)</span> == <span style="color:#f1bd47">TRUE</span> == <span style="color:#f1bd47">λx. λy. x</span>

In [15]:
def term_test_12():
    return multi_app_term(eq_term(), num_term(2), num_term(2))


test_procedure(term_test_12(), steps_lim=100)

0: (((λ#3473. (λ#3474. (((λ#3475. (λ#3476. ((((λ#3477. (λ#3478. (λ#3479. ((#3477 #3478) #3479)))) #3475) #3476) #3475))) (((λ#3480. (λ#3481. ((λ#3482. ((#3482 (λ#3483. (λ#3484. (λ#3485. #3485)))) (λ#3486. (λ#3487. #3486)))) (((λ#3488. (λ#3489. ((#3489 (λ#3490. (λ#3491. (λ#3492. (((#3490 (λ#3493. (λ#3494. (#3494 (#3493 #3491))))) (λ#3495. #3492)) (λ#3496. #3496)))))) #3488))) #3480) #3481)))) #3473) #3474)) (((λ#3497. (λ#3498. ((λ#3499. ((#3499 (λ#3500. (λ#3501. (λ#3502. #3502)))) (λ#3503. (λ#3504. #3503)))) (((λ#3505. (λ#3506. ((#3506 (λ#3507. (λ#3508. (λ#3509. (((#3507 (λ#3510. (λ#3511. (#3511 (#3510 #3508))))) (λ#3512. #3509)) (λ#3513. #3513)))))) #3505))) #3497) #3498)))) #3474) #3473)))) (λ#3514. (λ#3515. (#3514 (#3514 #3515))))) (λ#3516. (λ#3517. (#3516 (#3516 #3517)))))
count_redexes: 9
count_redexes: 9
1: ((λ#3518. (((λ#3519. (λ#3520. ((((λ#3521. (λ#3522. (λ#3523. ((#3521 #3522) #3523)))) #3519) #3520) #3519))) (((λ#3524. (λ#3525. ((λ#3526. ((#3526 (λ#3527. (λ#3528. (λ#3529. #35

# <span style="color: #A1F99C">13.</span>
<span style="color:#bb91f3">(EQ 1 4)</span> == <span style="color:#f1bd47">FALSE</span> == <span style="color:#f1bd47">λx. λy. y</span>

In [16]:
def term_test_13():
    return multi_app_term(eq_term(), num_term(1), num_term(4))


test_procedure(term_test_13(), steps_lim=100)

0: (((λ#5981. (λ#5982. (((λ#5983. (λ#5984. ((((λ#5985. (λ#5986. (λ#5987. ((#5985 #5986) #5987)))) #5983) #5984) #5983))) (((λ#5988. (λ#5989. ((λ#5990. ((#5990 (λ#5991. (λ#5992. (λ#5993. #5993)))) (λ#5994. (λ#5995. #5994)))) (((λ#5996. (λ#5997. ((#5997 (λ#5998. (λ#5999. (λ#6000. (((#5998 (λ#6001. (λ#6002. (#6002 (#6001 #5999))))) (λ#6003. #6000)) (λ#6004. #6004)))))) #5996))) #5988) #5989)))) #5981) #5982)) (((λ#6005. (λ#6006. ((λ#6007. ((#6007 (λ#6008. (λ#6009. (λ#6010. #6010)))) (λ#6011. (λ#6012. #6011)))) (((λ#6013. (λ#6014. ((#6014 (λ#6015. (λ#6016. (λ#6017. (((#6015 (λ#6018. (λ#6019. (#6019 (#6018 #6016))))) (λ#6020. #6017)) (λ#6021. #6021)))))) #6013))) #6005) #6006)))) #5982) #5981)))) (λ#6022. (λ#6023. (#6022 #6023)))) (λ#6024. (λ#6025. (#6024 (#6024 (#6024 (#6024 #6025)))))))
count_redexes: 9
count_redexes: 9
count_redexes: 8
count_redexes: 8
count_redexes: 8
count_redexes: 5
count_redexes: 3
count_redexes: 3
1: (((λ#6026. (λ#6027. (((λ#6028. (λ#6029. ((((λ#6030. (λ#6031. (λ#60